In [1]:
import pandas as pd
import numpy as np

In [2]:
raw = pd.read_csv('../data/qgis.csv', index_col=0)

In [3]:
raw.head()

,SpaceID,BlockFace,MeterType,RateType,RateRange,MeteredTimeLimit,ParkingPolicy,StreetCleaning,LatLng,Lat,Lng
0,VS336,4900 SUNSET BL,Single-Space,FLAT,$1.00,2HR,"TANS 4P-7P Mon-Fri | PKG 8A-4P Mon-Fri, 8A-8P Sat",NaN,"(34.098024, -118.296664)",34.098024,-118.296664
1,ED396,700 E PICO BLVD,Single-Space,TOD,$0.5 - $1,2HR,8A-6P Mon-Sat,NaN,"(34.032054, -118.252521)",34.032054,-118.252521
2,CB211A,1300 S GRAND AVE,Single-Space,FLAT,$0.50,4HR,8A-8P Mon-Sat,NaN,"(34.038125, -118.264675)",34.038125,-118.264675
3,SL408,3100 SUNSET BL,Single-Space,FLAT,$1.00,2HR,8A-8P Mon-Sat,NaN,"(34.084343, -118.274463)",34.084343,-118.274463
4,VN1,7001 VAN NUYS BL,Single-Space,FLAT,$1.00,2HR,8A-8P Mon-Sat,NaN,"(34.198035, -118.44891)",34.198035,-118.448910


In [4]:
grouped = raw.groupby(['BlockFace', 'MeterType','RateType','RateRange','MeteredTimeLimit','ParkingPolicy'])\
    .agg({'LatLng': lambda x : ', '.join(x)})
grouped.reset_index(inplace=True)

In [5]:
grouped['group'] = grouped.index

In [6]:
def rm_parentheses(s: str):
    return s.replace('(', '').replace(')', '').replace(' ', '')

def split_n(s, sep, n):
    s = s.split(sep)
    return [s[i:i+n] for i in range(0, len(s), n)]

def split_latlng(latlng: str):
    stripped = rm_parentheses(latlng)
    coords = split_n(stripped, ',', 2)
    return coords

In [7]:
grouped['coord_list'] = grouped['LatLng'].apply(split_latlng)

In [9]:
grouped = grouped.drop('LatLng', axis=1).explode('coord_list').reset_index(drop=True)
grouped['order'] = grouped.index

In [14]:
grouped['lat'] = [coord[0] for coord in grouped.coord_list]
grouped['long'] = [coord[1] for coord in grouped.coord_list]

In [15]:
grouped.head()

,BlockFace,MeterType,RateType,RateRange,MeteredTimeLimit,ParkingPolicy,group,coord_list,order,lat,long
0,1 NORTH VENICE BL (S),Single-Space,FLAT,$2.00,1HR,8A-6P Daily,0,"[33.984622, -118.470849]",0,33.984622,-118.470849
1,1 NORTH VENICE BL (S),Single-Space,FLAT,$2.00,1HR,8A-6P Daily,0,"[33.98493, -118.470343]",1,33.98493,-118.470343
2,1 NORTH VENICE BL (S),Single-Space,FLAT,$2.00,1HR,8A-6P Daily,0,"[33.984668, -118.470774]",2,33.984668,-118.470774
3,1 NORTH VENICE BL (S),Single-Space,FLAT,$2.00,1HR,8A-6P Daily,0,"[33.984961, -118.470293]",3,33.984961,-118.470293
4,1 NORTH VENICE BL (S),Single-Space,FLAT,$2.00,1HR,8A-6P Daily,0,"[33.984769, -118.470606]",4,33.984769,-118.470606


In [18]:
df = grouped.drop('coord_list', axis=1)

In [20]:
df.head()

,BlockFace,MeterType,RateType,RateRange,MeteredTimeLimit,ParkingPolicy,group,order,lat,long
0,1 NORTH VENICE BL (S),Single-Space,FLAT,$2.00,1HR,8A-6P Daily,0,0,33.984622,-118.470849
1,1 NORTH VENICE BL (S),Single-Space,FLAT,$2.00,1HR,8A-6P Daily,0,1,33.98493,-118.470343
2,1 NORTH VENICE BL (S),Single-Space,FLAT,$2.00,1HR,8A-6P Daily,0,2,33.984668,-118.470774
3,1 NORTH VENICE BL (S),Single-Space,FLAT,$2.00,1HR,8A-6P Daily,0,3,33.984961,-118.470293
4,1 NORTH VENICE BL (S),Single-Space,FLAT,$2.00,1HR,8A-6P Daily,0,4,33.984769,-118.470606


In [21]:
df.to_csv('../data/grouped_qgis.csv')